<a href="https://colab.research.google.com/github/PriyaMurali/Data-Analysis/blob/main/TelecomRDDLearningDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
!apt-get update
!apt-get install --fix-missing openjdk-8-jdk-headless -qq > /dev/null



Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Ge

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("PySparkETLTelecom") \
        .getOrCreate()
sc = spark.sparkContext

In [ ]:
#Read,Save,Filter RDD
valRDD = sc.textFile("/content/LexLearningTelecomData.csv")
RddExtract = valRDD.map(lambda var1: var1.split(","))
RddExtract.take(10)
FilteredRDD = RddExtract.filter(lambda r: (r[3] == "Y") & (r[9] == "Y") )
FilteredRDD.saveAsTextFile("/content/SeniorTelecomData.csv")
FilteredRDD


In [ ]:
#PariedRDD
PairedData = FilteredRDD.map(lambda x: (x[4],x[7]))
PairedData.take(10)


[('PrePaid', 'InActive'),
 ('PrePaid', 'Active'),
 ('PrePaid', 'Active'),
 ('PrePaid', 'Active'),
 ('PrePaid', 'Active'),
 ('PrePaid', 'Active'),
 ('PrePaid', 'Active'),
 ('PrePaid', 'Active'),
 ('PrePaid', 'Active'),
 ('PrePaid', 'Active')]

In [ ]:
rdd2=PairedData.countByValue()
rdd2

defaultdict(int,
            {('PostPaid', 'Active'): 28,
             ('PostPaid', 'InActive'): 1,
             ('PrePaid', 'Active'): 23,
             ('PrePaid', 'InActive'): 2})

In [ ]:
#Read,Save,Filter RDD
broadRDD = sc.textFile("/content/BroadCastData.csv")
bRd = broadRDD.map(lambda var1: var1.split(","))
bRd.take(10)


[['982120000', 'United States'],
 ['982120001', 'Australia'],
 ['982120002', 'Germany'],
 ['982120003', 'France'],
 ['982120004', 'India'],
 ['982120005', 'Canada'],
 ['982120006', 'Germany'],
 ['982120007', 'France'],
 ['982120008', 'India'],
 ['982120009', 'Canada']]

In [ ]:
#Join teo RDD
broadColumns = ["Customerid","Country"]
broadcastDF=spark.createDataFrame(bRd,broadColumns)
TelecomColumns = ["id","Customerid","Gender","SeniorStatus","SimType","S1","S2","S3","Age","SS"]
telecommDF=spark.createDataFrame(RddExtract,TelecomColumns)
joinedDF = telecommDF.join(broadcastDF,["Customerid"],how='left')
print(joinedDF.show())


+----------+---+------+------------+--------+------+------+--------+---+---+-------------+
|Customerid| id|Gender|SeniorStatus| SimType|    S1|    S2|      S3|Age| SS|      Country|
+----------+---+------+------------+--------+------+------+--------+---+---+-------------+
| 982120011| 12|Female|           N|PostPaid|Active|Active|InActive| 20|  N|       France|
| 982120016| 17|Female|           N|PostPaid|Active|Active|InActive| 15|  N|      Germany|
| 982120010| 11|Female|           N| PrePaid|Active|Active|InActive| 15|  N|      Germany|
| 982120017| 18|Female|           N|PostPaid|Active|Active|InActive| 15|  N|       France|
| 982120009| 10|Female|           N| PrePaid|Active|Active|InActive| 15|  N|       Canada|
| 982120008|  9|  Male|           Y| PrePaid|Active|Active|InActive| 15|  Y|        India|
| 982120012| 13|Female|           N|PostPaid|Active|Active|InActive| 25|  Y|        India|
| 982120003|  4|  Male|           Y| PrePaid|Active|Active|InActive| 25|  N|       France|

In [137]:
joinedDF.write.csv("/content/SeniorTelecomData/TextFile.csv",header=True)
joinedDF

DataFrame[Customerid: string, id: string, Gender: string, SeniorStatus: string, SimType: string, S1: string, S2: string, S3: string, Age: string, SS: string, Country: string]